In [10]:
import json 
import os 
import pandas as pd 


In [11]:
output_folder_path = '/Users/rt853/repos/UoB/bath-persuasion-detection/data/phase_2_data/output_from_doccano/output_phase_2_partial_complete'


In [12]:
def read_jsonl_file(file_path, user):
    processed_list = []
    with open(file_path) as f:
        
        jline = f.readlines()
        
        for idx in range(len(jline)):
            j = json.loads(jline[idx])
            j['page'] =  int(j['text'].split(' ')[1])
            j['user'] = user
            for i in j['label']:
                i.append(user)
            j['num_labels'] = len(j['label'])
            if len(j['label']) < 1:
                continue 
            else:
                processed_list.append(j)
    
    return processed_list

def process_jsonlists(output_folder_path):
    dataset = []           
    for file in os.listdir(output_folder_path):
        path = output_folder_path + '/' + file

        if path.endswith('.jsonl'):
            d_list = []
            user = file.split('.')[0]
            jsonl = read_jsonl_file(path, file)
            dataset.extend(jsonl)
    return dataset 

dataset = process_jsonlists(output_folder_path)
df = pd.DataFrame(dataset)



In [13]:
def process_annotation(df, page_num=1):

    dataset = []
    page_1 = df[df.page == page_num]

    for idx,  row in page_1.iterrows():
        
        text = row.text

        for start_idx, end_idx, label, user in row.label:
            annotation = {}
            annotation['text'] = text[start_idx:end_idx]
            annotation['start_idx'] = start_idx
            annotation['end_idx'] = end_idx
            annotation['top_level'] = '-'.join(label.split('-')[:-1])
            annotation['bottom_level'] = label.split('-')[-1]
            annotation['label'] = label
            annotation['user'] = user.split('.')[0]
            annotation['page'] = row.page
            dataset.append(annotation)
            
    return dataset

dataset = process_annotation(df, page_num=1)
d2 = pd.DataFrame(dataset).reset_index(drop=True).sort_values(by='start_idx')


In [14]:
def init_users():
    users = ['Sara', 'Nadeen', 'Adela', 'Dessi','Samiksha', 'Jessie']
    user_ds = []
    for user in users:
        user_d = d2[d2['user'] == user]   
        user_ds.append(user_d)
    return user_ds

# users = init_users()


In [15]:
def process_annotations(annotations):

    labels = []
    for i in annotations:
        text, label, user = i
        user = user.split('.')[0]
        # label = '-'.join(label.split('-')[:2])
        if (label,user) not in labels:
            labels.append((label, user))

    labels.sort(key=lambda x: x[1])
    return labels


In [16]:
def process_annotated_lines(annotated_lines, d):
    
    for i in annotated_lines:
        
        users = { 'Adela' : [],
            'Dessi' : [],
            'Jessie' : [],
            'Nadeen' : [],
            'Samiksha' : [],
            'Sara' : []}
        
        if len(i) < 2:
            continue 
        
        text, annotations = i
        processed_annotations = process_annotations(annotations)
        d['text'].append(text)
        
        for label, user in processed_annotations:
            if user not in users:
                continue
            users[user].append(label)
            
        merged = {user : '/'.join(labels) for user, labels in users.items()}
        
        for k, v in merged.items():
            if len(v) == 0:
                v = '8-NO-PERSUASION'
            d[k].append(v)
    return d 


In [17]:
def annotate_string(long_string, labels):
    # Split the long string by new lines to create the list of lines
    lines = long_string.split('\n')
    
    # Create a list to hold the annotated lines
    annotated_lines = []

    # Flatten the labels list with their original indexes
    all_labels = []
    for start, stop, label, user in labels:
        all_labels.append((start, stop, label, user))
    
    # Sort labels based on the start index
    all_labels.sort(key=lambda x: x[0])

    # Iterate over each line
    current_pos = 0
    for line in lines:
        line_length = len(line)
        line_labels = []

        # Iterate over each label and check if it falls within the current line
        while all_labels and all_labels[0][0] < current_pos + line_length:
            start, stop, label, user = all_labels.pop(0)
            
            # Calculate the relative start and stop positions within the current line
            rel_start = max(0, start - current_pos)
            rel_stop = min(line_length, stop - current_pos)
            
            # Append the labeled portion if within current line
            if rel_start < line_length and rel_stop > 0:
                line_labels.append((line[rel_start:rel_stop], label, user))
        
        # Create the annotated line as ['text', labels]
        annotated_lines.append([line, line_labels])
        
        # Update the current position to the start of the next line
        current_pos += line_length + 1  # +1 for the newline character

    return annotated_lines

d = {
    'text' : [],
    'Adela' : [],
    'Dessi' : [],
    'Jessie' : [],
    'Nadeen' : [],
    'Samiksha' : [],
    'Sara' : []
}

for page_num in range(len(df.page.unique())):
    page_num+=1
    text = df[df['page']== page_num].iloc[0].text
    
    labels = []
    for i in df[df['page'] == page_num].label.tolist():
        labels += i

    annotated_lines = annotate_string(text, labels)
    d = process_annotated_lines(annotated_lines, d)
    
df4 = pd.DataFrame(d)


In [19]:
df4 = df4.dropna(subset=['text'])



In [24]:

for i, row in df4.iterrows():
    if len(row.text) < 5:
        df4.drop(i, inplace=True)


In [25]:
df4


,text,Adela,Dessi,Jessie,Nadeen,Samiksha,Sara
0,Page 1 of 41,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION
1,"Persuader: Hello gorgeous ,I’m *name* from cal...",1-RAPPORT-FRIEND/1-RAPPORT-SIMILAR/1-RAPPORT-D...,1-RAPPORT-COMPLIMENT,1-RAPPORT-COMPLIMENT/1-RAPPORT-DISCLOSE,1-RAPPORT-COMPLIMENT,1-RAPPORT-FRIEND/1-RAPPORT-COMPLIMENT,1-RAPPORT-OTHER/1-RAPPORT-COMPLIMENT
2,Victim: Hi *name*. Where are you from?,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION
3,Persuader: I live in *location,1-RAPPORT-DISCLOSE,1-RAPPORT-DISCLOSE,1-RAPPORT-DISCLOSE,8-NO-PERSUASION,8-NO-PERSUASION,1-RAPPORT-OTHER
4,"Persuader: I'm presently at *location*, Workin...",1-RAPPORT-DISCLOSE/5-AUTHORITY-CREDENTIALS,1-RAPPORT-DISCLOSE/5-AUTHORITY-CREDENTIALS,1-RAPPORT-DISCLOSE/5-AUTHORITY-SUPERIOR,1-RAPPORT-DISCLOSE,1-RAPPORT-FRIEND,1-RAPPORT-DISCLOSE
...,...,...,...,...,...,...,...
896,Victim: To your hangout?,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION
897,Persuader: Yes,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION
898,Victim: I thought it wouldnt work unless I acc...,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION,8-NO-PERSUASION
899,Persuader: No it would work babe,1-RAPPORT-FRIEND,8-NO-PERSUASION,4-LOGIC-COUNTER,8-NO-PERSUASION,8-NO-PERSUASION,4-LOGIC-OTHER
